In [ ]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz


In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession
 .builder
 .appName("modelx")
 .getOrCreate())


In [ ]:
import gdown
url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ?usp=sharing"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/chess/chess_schema.png',
 '/content/chess/chess_wc_history_game_info.csv',
 '/content/chess/chess_wc_history_moves.csv',
 '/content/chess/eco_codes.csv']

In [ ]:
eco_codes_df=spark.read.option("header","true").option("inferSchema","true").csv("/content/chess/eco_codes.csv")

In [ ]:
eco_codes_df.show(10)

+---+--------------------+--------------------+--------+---------+
|eco|            eco_name|         eco_example|eco_type|eco_group|
+---+--------------------+--------------------+--------+---------+
|A00|    Uncommon Opening|  1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|                1 b3|       A|    Flank|
|A02|      Bird's Opening|                1 f4|       A|    Flank|
|A03|      Bird's Opening|             1 f4 d5|       A|    Flank|
|A04|        Reti Opening|               1 Nf3|       A|    Flank|
|A05|        Reti Opening|           1 Nf3 Nf6|       A|    Flank|
|A06|        Reti Opening|            1 Nf3 d5|       A|    Flank|
|A07|King's Indian Attack|       1 Nf3 d5 2 g3|       A|    Flank|
|A08|King's Indian Attack|1 Nf3 d5 2 g3 c5 ...|       A|    Flank|
|A09|        Reti Opening|       1 Nf3 d5 2 c4|       A|    Flank|
+---+--------------------+--------------------+--------+---------+
only showing top 10 rows



In [ ]:
chess_wc_history_moves_df=spark.read.option("header","true").option("inferSchema","true").csv("/content/chess/chess_wc_history_moves.csv")

In [ ]:
chess_wc_history_moves_df.show(10)

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
chess_wc_history_game_info_df=spark.read.option("header","true").option("inferSchema","true").csv("/content/chess/chess_wc_history_game_info.csv")

In [ ]:
chess_wc_history_game_info_df.show(10)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|  1.0|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            draw|  

In [ ]:
chess_wc_history_game_info_df.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- game_order: integer (nullable = true)
 |-- event: string (nullable = true)
 |-- site: string (nullable = true)
 |-- date_played: string (nullable = true)
 |-- round: double (nullable = true)
 |-- white: string (nullable = true)
 |-- black: string (nullable = true)
 |-- result: string (nullable = true)
 |-- white_elo: integer (nullable = true)
 |-- black_elo: integer (nullable = true)
 |-- white_title: string (nullable = true)
 |-- black_title: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- winner_elo: integer (nullable = true)
 |-- loser: string (nullable = true)
 |-- loser_elo: integer (nullable = true)
 |-- winner_loser_elo_diff: integer (nullable = true)
 |-- eco: string (nullable = true)
 |-- date_created: string (nullable = true)
 |-- tournament_name: string (nullable = true)



In [ ]:
eco_codes_df.createTempView("eco_codes")

In [ ]:
spark.sql("SELECT * FROM eco_codes LIMIT 5").show()

+---+--------------------+------------------+--------+---------+
|eco|            eco_name|       eco_example|eco_type|eco_group|
+---+--------------------+------------------+--------+---------+
|A00|    Uncommon Opening|1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|              1 b3|       A|    Flank|
|A02|      Bird's Opening|              1 f4|       A|    Flank|
|A03|      Bird's Opening|           1 f4 d5|       A|    Flank|
|A04|        Reti Opening|             1 Nf3|       A|    Flank|
+---+--------------------+------------------+--------+---------+



In [ ]:
chess_wc_history_moves_df.createTempView("chess_wc_history_moves")

In [ ]:
spark.sql("SELECT * FROM chess_wc_history_moves LIMIT 5").show()

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
chess_wc_history_game_info_df.createTempView("chess_wc_history_game_info")

In [ ]:
df1=spark.sql("""SELECT DISTINCT winner,tournament_name 
              FROM chess_wc_history_game_info WHERE winner!='draw' AND tournament_name LIKE 'World%' 
              GROUP BY tournament_name,winner ORDER BY tournament_name""")

In [ ]:
df1.show()

+--------------------+---------------+
|              winner|tournament_name|
+--------------------+---------------+
|   Steinitz, William| WorldChamp1886|
|Zukertort, Johann...| WorldChamp1886|
|   Steinitz, William| WorldChamp1889|
|   Chigorin, Mikhail| WorldChamp1889|
|    Gunsberg, Isidor| WorldChamp1890|
|   Steinitz, William| WorldChamp1890|
|   Chigorin, Mikhail| WorldChamp1892|
|   Steinitz, William| WorldChamp1892|
|     Lasker, Emanuel| WorldChamp1894|
|   Steinitz, William| WorldChamp1894|
|   Steinitz, William| WorldChamp1896|
|     Lasker, Emanuel| WorldChamp1896|
|     Lasker, Emanuel| WorldChamp1907|
|  Tarrasch, Siegbert| WorldChamp1908|
|     Lasker, Emanuel| WorldChamp1908|
|Janowsky, Dawid M...| WorldChamp1909|
|     Lasker, Emanuel| WorldChamp1909|
|    Schlechter, Carl|WorldChamp1910a|
|     Lasker, Emanuel|WorldChamp1910a|
|     Lasker, Emanuel|WorldChamp1910b|
+--------------------+---------------+
only showing top 20 rows



In [ ]:
df2=spark.sql("""SELECT DISTINCT winner,tournament_name, COUNT(winner) as number_of_wins
              FROM chess_wc_history_game_info WHERE winner!='draw' 
              GROUP BY tournament_name, winner ORDER BY number_of_wins DESC""")

In [ ]:
df2.show()

+--------------------+---------------+--------------+
|              winner|tournament_name|number_of_wins|
+--------------------+---------------+--------------+
|      Kasimdzhanov,R|  FideChamp2004|            13|
|        Ponomariov,R|  FideChamp2002|            12|
|Khalifman, Alexander|  FideChamp1999|            12|
| Alekhine, Alexander| WorldChamp1929|            11|
|   Steinitz, William| WorldChamp1886|            10|
| Alekhine, Alexander| WorldChamp1937|            10|
|   Steinitz, William| WorldChamp1892|            10|
|  Anand, Viswanathan|  FideChamp1998|            10|
|   Steinitz, William| WorldChamp1889|            10|
|     Lasker, Emanuel| WorldChamp1894|            10|
|  Botvinnik, Mikhail| WorldChamp1961|            10|
|  Botvinnik, Mikhail| WorldChamp1948|            10|
|     Lasker, Emanuel| WorldChamp1896|            10|
|            Adams,Mi|  FideChamp2004|             9|
|      Short, Nigel D|  FideChamp1998|             9|
|           Euwe, Max| World

In [ ]:
df3=spark.sql("""(SELECT chess.eco, COUNT(chess.eco) AS number_of_occurences, eco_name 
            FROM chess_wc_history_game_info chess INNER JOIN eco_codes codes ON codes.eco=chess.eco 
            GROUP BY chess.eco, codes.eco_name 
            UNION
            SELECT chess.eco, COUNT(chess.eco) AS number_of_occurences, eco_name 
            FROM chess_wc_history_game_info chess INNER JOIN eco_codes codes ON codes.eco=chess.eco 
            GROUP BY chess.eco, codes.eco_name
            ORDER BY number_of_occurences DESC LIMIT 1)
            UNION
            (SELECT chess.eco, COUNT(chess.eco) AS number_of_occurences, eco_name 
            FROM chess_wc_history_game_info chess INNER JOIN eco_codes codes ON codes.eco=chess.eco 
            GROUP BY chess.eco, codes.eco_name 
            UNION
            SELECT chess.eco, COUNT(chess.eco) AS number_of_occurences, eco_name 
            FROM chess_wc_history_game_info chess INNER JOIN eco_codes codes ON codes.eco=chess.eco 
            GROUP BY chess.eco, codes.eco_name
            ORDER BY number_of_occurences LIMIT 1)""")

In [ ]:
df3.show()

+---+--------------------+--------------------+
|eco|number_of_occurences|            eco_name|
+---+--------------------+--------------------+
|E30|                   1|Nimzo-Indian, Len...|
|C42|                  64|      Petrov Defense|
+---+--------------------+--------------------+



In [ ]:
df4=spark.sql("""SELECT chess.eco, COUNT(chess.winner) AS number_of_wins, eco_name 
            FROM chess_wc_history_game_info chess INNER JOIN eco_codes codes ON codes.eco=chess.eco 
            WHERE chess.winner!='draw' GROUP BY chess.eco, codes.eco_name 
            ORDER BY number_of_wins DESC LIMIT 1""")

In [ ]:
df4.select("eco","eco_name").show()

+---+-----------------+
|eco|         eco_name|
+---+-----------------+
|B90|Sicilian, Najdorf|
+---+-----------------+



In [ ]:
df5=spark.sql("""(SELECT moves.game_id,chess.event,chess.tournament_name,MAX(moves.move_no_pair) AS move_count 
        FROM chess_wc_history_game_info chess INNER JOIN chess_wc_history_moves moves ON moves.game_id=chess.game_id 
        GROUP BY moves.game_id,chess.event, chess.tournament_name ORDER BY move_count DESC LIMIT 1)
        UNION
        (SELECT moves.game_id,chess.event,chess.tournament_name,MAX(moves.move_no_pair) AS move_count 
        FROM chess_wc_history_game_info chess INNER JOIN chess_wc_history_moves moves ON moves.game_id=chess.game_id 
        GROUP BY moves.game_id,chess.event, chess.tournament_name ORDER BY move_count ASC LIMIT 1)""")

In [ ]:
df5.show()

+--------------------+--------------------+---------------+----------+
|             game_id|               event|tournament_name|move_count|
+--------------------+--------------------+---------------+----------+
|4424a0a4-3732-407...|         FIDE WCh KO|  FideChamp2000|       146|
|1846cede-0037-4f0...|World Championshi...| WorldChamp1972|         1|
+--------------------+--------------------+---------------+----------+



In [ ]:
df6=spark.sql("""(SELECT moves.game_id,chess.event,chess.tournament_name,MAX(moves.move_no_pair) AS number_of_moves 
        FROM chess_wc_history_game_info chess INNER JOIN chess_wc_history_moves moves ON moves.game_id=chess.game_id 
        WHERE chess.winner='draw'
        GROUP BY moves.game_id,chess.event, chess.tournament_name ORDER BY number_of_moves DESC LIMIT 1)
        UNION
        (SELECT moves.game_id,chess.event,chess.tournament_name,MAX(moves.move_no_pair) AS number_of_moves 
        FROM chess_wc_history_game_info chess INNER JOIN chess_wc_history_moves moves ON moves.game_id=chess.game_id 
        WHERE chess.winner='draw'
        GROUP BY moves.game_id,chess.event, chess.tournament_name ORDER BY number_of_moves ASC LIMIT 1)""")

In [ ]:
df6.show()

+--------------------+-------------+---------------+---------------+
|             game_id|        event|tournament_name|number_of_moves|
+--------------------+-------------+---------------+---------------+
|88f34084-e4df-490...|  FIDE WCh KO|  FideChamp2002|            129|
|a27c2b95-fb62-4b6...|FIDE-Wch k.o.|  FideChamp1998|              6|
+--------------------+-------------+---------------+---------------+



In [ ]:
df7=spark.sql("""select DISTINCT winner as player_name, winner_elo as elo
            from chess_wc_history_game_info
            where winner_elo = (select max(winner_elo) from chess_wc_history_game_info)
            union all
            select DISTINCT loser as player_name, loser_elo as elo
            from chess_wc_history_game_info
            where loser_elo = (select min(loser_elo) from chess_wc_history_game_info)""")  

In [ ]:
df7.show()

+---------------+----+
|    player_name| elo|
+---------------+----+
|Carlsen, Magnus|2870|
|      Abulhul,T|2076|
+---------------+----+



In [ ]:
df8=spark.sql("""SELECT * FROM 
            (SELECT loser as player_name, COUNT(loser) AS number_of_losses 
            FROM chess_wc_history_game_info WHERE winner !='draw' 
            GROUP BY loser ORDER BY number_of_losses DESC LIMIT 3) 
            ORDER BY number_of_losses LIMIT 1""")

In [ ]:
df8.show()

+---------------+----------------+
|    player_name|number_of_losses|
+---------------+----------------+
|Karpov, Anatoly|              36|
+---------------+----------------+



In [ ]:
df9=spark.sql("""SELECT DISTINCT winner AS player_name, COUNT(*) AS win_count 
            FROM chess_wc_history_game_info 
            WHERE winner_loser_elo_diff<0 
            GROUP BY winner ORDER BY win_count DESC""")

In [ ]:
df9.show()

+--------------------+---------+
|         player_name|win_count|
+--------------------+---------+
|     Karpov, Anatoly|       20|
|      Kasimdzhanov,R|       11|
|           Macieja,B|        9|
|  Kortschnoj, Viktor|        8|
|           Kramnik,V|        8|
|          Grischuk,A|        7|
|        Ponomariov,R|        7|
|Khalifman, Alexander|        6|
|      Kasparov, Gary|        5|
|   Akopian, Vladimir|        5|
|           Sulskis,S|        5|
|      Short, Nigel D|        5|
|         Dominguez,L|        5|
|           Ehlvest,J|        5|
|           Gelfand,B|        4|
|Nisipeanu, Liviu ...|        4|
|         Filippov,Va|        4|
|          Nakamura,H|        4|
| Tkachiev, Vladislav|        4|
|            Volkov,S|        4|
+--------------------+---------+
only showing top 20 rows



In [ ]:
df10=spark.sql("""SELECT DISTINCT game_id, player as player_name, move_sequence, move_no_pair as move_count 
        FROM chess_wc_history_moves 
        GROUP BY game_id,player,move_sequence,move_count ORDER BY game_id,move_count ASC""")

In [ ]:
df10.show()

+--------------------+-----------+--------------------+----------+
|             game_id|player_name|       move_sequence|move_count|
+--------------------+-----------+--------------------+----------+
|0028cc2e-5aaa-464...|Khalifman,A|                  d4|         1|
|0028cc2e-5aaa-464...|     Leko,P|              d4|Nf6|         1|
|0028cc2e-5aaa-464...|Khalifman,A|          d4|Nf6|Nf3|         2|
|0028cc2e-5aaa-464...|     Leko,P|       d4|Nf6|Nf3|g6|         2|
|0028cc2e-5aaa-464...|Khalifman,A|    d4|Nf6|Nf3|g6|c4|         3|
|0028cc2e-5aaa-464...|     Leko,P|d4|Nf6|Nf3|g6|c4|Bg7|         3|
|0028cc2e-5aaa-464...|Khalifman,A|d4|Nf6|Nf3|g6|c4|...|         4|
|0028cc2e-5aaa-464...|     Leko,P|d4|Nf6|Nf3|g6|c4|...|         4|
|0028cc2e-5aaa-464...|Khalifman,A|d4|Nf6|Nf3|g6|c4|...|         5|
|0028cc2e-5aaa-464...|     Leko,P|d4|Nf6|Nf3|g6|c4|...|         5|
|0028cc2e-5aaa-464...|     Leko,P|d4|Nf6|Nf3|g6|c4|...|         6|
|0028cc2e-5aaa-464...|Khalifman,A|d4|Nf6|Nf3|g6|c4|...|       

In [ ]:
df11=spark.sql("""SELECT COUNT(loser) as total_number_of_games FROM chess_wc_history_game_info WHERE loser!='draw'
            UNION
            (SELECT player FROM chess_wc_history_moves WHERE color='white' AND captured_score_for_white>captured_score_for_black)
            UNION
            (SELECT player FROM chess_wc_history_moves WHERE color='black' AND captured_score_for_black>captured_score_for_white)""")

In [ ]:
df11.show()

+---------------------+
|total_number_of_games|
+---------------------+
|                 1423|
+---------------------+



In [ ]:
df12=spark.sql("""SELECT winner,tournament_name 
            FROM chess_wc_history_game_info 
            WHERE winner!=loser AND loser!=winner 
            GROUP by tournament_name,winner 
            ORDER BY tournament_name DESC""")

In [ ]:
df12.show()

+------------------+---------------+
|            winner|tournament_name|
+------------------+---------------+
|         Carlsen,M| WorldChamp2021|
|   Carlsen, Magnus| WorldChamp2016|
|  Karjakin, Sergey| WorldChamp2016|
|   Carlsen, Magnus| WorldChamp2014|
|Anand, Viswanathan| WorldChamp2014|
|   Carlsen, Magnus| WorldChamp2013|
|         Gelfand,B| WorldChamp2012|
|Anand, Viswanathan| WorldChamp2012|
|           Anand,V| WorldChamp2012|
|         Topalov,V| WorldChamp2010|
|           Anand,V| WorldChamp2010|
|           Anand,V| WorldChamp2008|
|         Kramnik,V| WorldChamp2008|
|         Svidler,P| WorldChamp2007|
|         Gelfand,B| WorldChamp2007|
|           Anand,V| WorldChamp2007|
|         Kramnik,V| WorldChamp2007|
|      Morozevich,A| WorldChamp2007|
|            Leko,P| WorldChamp2007|
|        Grischuk,A| WorldChamp2007|
+------------------+---------------+
only showing top 20 rows



In [ ]:
df13=spark.sql("""select *, win_count/game_count as win_ratio 
            from (Select winner,win_count 
            from (select count(1) as win_count, winner 
            from chess_wc_history_game_info where winner!='draw' group by winner) a join (Select distinct player_name 
            from ((Select distinct black as player_name from chess_wc_history_game_info) 
            Union (Select distinct white as player_name from chess_wc_history_game_info))) b on a.winner=b.player_name) c 
            join (select a.player_name, black_game_count+white_game_count as game_count 
            from (select black as player_name, count(game_id) as black_game_count 
            from (Select distinct black, game_id from chess_wc_history_game_info) 
            group by black) a join (select white as player_name, count(game_id) as white_game_count 
            from (Select distinct white, game_id from chess_wc_history_game_info) 
            group by white) b on a.player_name=b.player_name) d on c.winner=d.player_name order by win_ratio desc limit 1""")

In [ ]:
df13.select("player_name").show()

+-----------+
|player_name|
+-----------+
| Kiseleva,N|
+-----------+



In [ ]:
df14=spark.sql("""SELECT player AS player_name FROM chess_wc_history_moves WHERE piece='P' AND is_check_mate=1""")

In [ ]:
df14.show()

+--------------+
|   player_name|
+--------------+
|Andersson, Ulf|
+--------------+



In [ ]:
df15=spark.sql("""select * from (select a.game_id, color, 
case when color='White' then white_queen_count
else black_queen_count end as queen_count,b.event,b.winner as player_name from
(select game_id, player, color, white_queen_count, black_queen_count from chess_wc_history_moves where is_game_over=1) a
join 
(select winner,game_id,event
 from chess_wc_history_game_info where winner!='draw') b
on a.game_id=b.game_id) where queen_count=0""")

In [ ]:
df15.select("game_id","event","player_name").show()

+--------------------+-------------+--------------+
|             game_id|        event|   player_name|
+--------------------+-------------+--------------+
|04c81fe4-efd8-456...|  FIDE WCh KO|    Nakamura,H|
|ac45a5ed-172d-4ae...|FIDE-Wch k.o.|Andersson, Ulf|
+--------------------+-------------+--------------+



In [ ]:
import os
directory="results"
parent_dir="/content/drive/MyDrive/DE_SOLUTION_Shrey_Ratna"
path = os.path.join(parent_dir,directory)
os.mkdir(path)
print("Directory '% s' created" % directory)

FileExistsError: ignored

In [ ]:
import pyspark
import pandas as pd
from pathlib import Path 
dfs=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]
filepath = Path('/content/drive/MyDrive/DE_SOLUTION_Shrey_Ratna/results/df.csv')
filepath.parent.mkdir(parents=True, exist_ok=True) 
for x in dfs:
    y=x.toPandas()
    y.to_csv(f"/content/drive/MyDrive/DE_SOLUTION_Shrey_Ratna/results/df{dfs.index(x) + 1}.csv", encoding='utf-8', index=False)